<a href="https://colab.research.google.com/github/ArtemBaskal/ML/blob/main/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HSE AML Homework 2 (2022)
https://www.kaggle.com/competitions/hse-aml-2022/overview

In [ ]:
import io
import pandas as pd
from google.colab import files
uploaded = files.upload()

Saving books_test.csv to books_test (2).csv
Saving books_train.csv to books_train (3).csv


In [ ]:
df = pd.read_csv(io.StringIO(uploaded['books_train.csv'].decode('utf-8')), error_bad_lines=False)
df_test = pd.read_csv(io.StringIO(uploaded['books_test.csv'].decode('utf-8')), error_bad_lines=False)
bookIDs_test = df_test.bookID.tolist()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
from sklearn.preprocessing import OrdinalEncoder

encoding = {'language_code':{'en-US': 'eng', 'en-GB': 'eng', 'en-CA': 'eng'}}
df.replace(encoding, inplace=True)
df_test.replace(encoding, inplace=True)

enc = OrdinalEncoder()
enc.fit(df[['language_code']])
df[['language_code']] = enc.fit_transform(df[['language_code']])

enc_test = OrdinalEncoder()
enc_test.fit(df_test[['language_code']])
df_test[['language_code']] = enc_test.fit_transform(df_test[['language_code']])

In [ ]:
df['publication_date'] = pd.to_datetime(df['publication_date'], format='%m/%d/%Y', errors='coerce')
df[df['publication_date'].isnull()]

df_test['publication_date'] = pd.to_datetime(df_test['publication_date'], format='%m/%d/%Y', errors='coerce')
df_test[df_test['publication_date'].isnull()]

,bookID,title,authors,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher


In [ ]:
df.loc[df.bookID == 31373, 'publication_date'] = '1999-10-01 00:00:00'
df.loc[df.bookID == 45531, 'publication_date'] = '1975-10-01 00:00:00'

In [ ]:
df['year'] = pd.DatetimeIndex(df['publication_date']).year

df.rename(columns = {'  num_pages': 'num_pages'}, inplace=True)

df_test['year'] = pd.DatetimeIndex(df_test['publication_date']).year

df_test.rename(columns = {'  num_pages': 'num_pages'}, inplace=True)

In [ ]:
df['num_occ'] = df.groupby('title')['title'].transform('count')

df_test['num_occ'] = df_test.groupby('title')['title'].transform('count')

In [ ]:
label = df['average_rating'].values
df.drop(['bookID', 'title', 'authors', 'isbn', 'isbn13', 'publication_date', 'publisher', 'average_rating'], axis=1, inplace=True)

df_test.drop(['bookID', 'title', 'authors', 'isbn', 'isbn13', 'publication_date', 'publisher'], axis=1, inplace=True)

In [ ]:
X_train, X_test, y_train = df, df_test, label

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

model = RandomForestRegressor()

parameters = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [3, 5, 7, 10, 12, 15],
    'min_samples_split': [5, 10, 15],
    'min_samples_leaf': [5, 10, 15]
}

grad_rf = GridSearchCV(model, parameters, refit=True, cv=10)
grad_rf.fit(X_train, y_train)

In [ ]:
# Random Forest Model
pred_rf = grad_rf.predict(X_test)
pred = pd.DataFrame({'bookID': bookIDs_test, 'average_rating': pred_rf.tolist()})

pred.to_csv('test.csv', index=False)
files.download('test.csv')